In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
pd.options.display.float_format = '{:,.0f}'.format
import glob

Carga de los archivos

In [2]:
lista = lista = glob.glob("D:/Zaeli/Analise Cliente Produto Comercial/Consolidado/Recientes/*.csv")
ventas = []

In [3]:
for archivo in lista:
    data = pd.read_csv(archivo, header=0, sep = ';', parse_dates = ['Ano/Mes'], dtype = {'CodGer':str,'CODSAP':str ,'CodRepresentante':str, 'Cli&Rep':str}, dayfirst=True)
    data = data.rename(columns={'Ano/Mes':'Fecha', 'CODSAP' : 'Material'})
    ventas.append(data)
ventas = pd.concat(ventas, ignore_index = True)

Preparacion de la base de datos comercial

In [4]:
inflacion = pd.read_excel("D:/Zaeli/BI/Archivos Compartidos/Inflacion.xlsx", header=3, sheet_name = 'Hoja1')
inflacion = inflacion.rename(columns= {'DataKey':'ChaveData'})
ventasi = pd.merge(ventas, inflacion)
ventasi['ReceitaBrutaIn'] = ventasi.ReceitaBruta * (1 + ventasi.InfacaoAcum)
ventasi['Verbas'] = ventasi.Bonif + ventasi.DescConcedidos + ventasi.Impostos + ventasi.Frete + ventasi.Comis - ventasi.Devol - ventasi.Troca
ventasi['ReceitaLiquida'] = ventasi.ReceitaBruta - ventasi.Verbas
ventasi['ReceitaLiquidaIn'] = ventasi.ReceitaLiquida * (1 + ventasi.InfacaoAcum)
ventasi['MargenBruto'] = ventasi.ReceitaLiquida - ventasi.CPV
ventasi['MargenBrutoIn'] = ventasi.MargenBruto * (1 + ventasi.InfacaoAcum)
ventasi = ventasi.drop(['PesoBruto', 'ReceitaBruta', 'Bonif', 'DescConcedidos', 'Impostos', 'Comis', 'CPV', 'Devol', 'Troca',                                                                       'Inflacaomes','InfacaoAcum', 'Verbas', 'MargenBruto', 'ReceitaLiquida'], axis=1)
ventasi = ventasi.rename(columns={
    'ReceitaBrutaIn' : 'ReceitaBruta',
    'ReceitaLiquidaIn' : 'ReceitaLiquida',
    'MargenBrutoIn' : 'Margen'
})

In [5]:
lineas = pd.read_csv("D:/Zaeli/PrevisionDemanda/lineasProducto.csv", header=0, sep=';')
lineas = lineas.rename(columns={
    'COD SAP':'Material',
    'Linha':'Linea',
    'Produto':'Producto'
})
lineas['Material'] = lineas['Material'].astype(str)
ventasP = pd.merge(ventasi, lineas)
ventasP = ventasP.drop(['Cli&Rep', 'Tipo'], axis=1)

In [6]:
ListadoClientes = pd.read_excel("D:/Zaeli/BI/Archivos Compartidos/Clientes.xlsx",sheet_name='Listado')
ListadoClientes = ListadoClientes.drop(['Cidade', 'CodEstado', 'Estado', 'CidadeEstado'], axis=1)
ventasC = pd.merge(ventasP, ListadoClientes)

In [8]:
ventasLM = ventasC.groupby([pd.Grouper(key='Fecha', freq='M'), 'Fecha', 'CodigoMunicipio', 'Linea'], as_index=False).agg({
    'ReceitaBruta':'sum',
    'ReceitaLiquida':'sum',
    'Volume':'sum',
    'PesoLiq':'sum',
    'Margen':'sum'
})

Carga y preparacion del archivo de los municipios

In [9]:
datosMun = pd.read_csv("D:/Zaeli/PrevisionDemanda/DatosMunicipios.csv", header=0, sep = ';')
datosMun = datosMun.rename(columns = {
    'CódigoMunicípio':'CodigoMunicipio',
    'Área Territorial 0 km² [2020]':'Area_km2',
    'População estimada 0 pessoas [2021]': 'Poblacion',
    'Densidade demográfica 0 hab/km² [2010]': 'Habitantes_km2',
    'Escolarização6a14%': 'PorcEscolarizacion6a14',
    'IDHM ' : 'IDHM',
    'Mortalidade infantil' : 'Mortalidad_Infantil'
})

In [10]:
datosMun.loc[datosMun['IDHM'] < 0.49 , 'Desarrollo'] = 'Muy Bajo'
datosMun.loc[(datosMun['IDHM'] >= 0.49) & (datosMun['IDHM'] < 0.599) , 'Desarrollo'] = 'Bajo'
datosMun.loc[(datosMun['IDHM'] >= 0.599) & (datosMun['IDHM'] < 0.699) , 'Desarrollo'] = 'Medio'
datosMun.loc[(datosMun['IDHM'] >= 0.699) & (datosMun['IDHM'] <= 0.799) , 'Desarrollo'] = 'Alto'
datosMun.loc[datosMun['IDHM'] >= 0.80 , 'Desarrollo'] = 'Muy Alto'
datosMun.loc[datosMun['IDHM'] < 0.49 , 'DesarrolloNum'] = '1'
datosMun.loc[(datosMun['IDHM'] >= 0.49) & (datosMun['IDHM'] < 0.599) , 'DesarrolloNum'] = '2'
datosMun.loc[(datosMun['IDHM'] >= 0.599) & (datosMun['IDHM'] < 0.699) , 'DesarrolloNum'] = '3'
datosMun.loc[(datosMun['IDHM'] >= 0.699) & (datosMun['IDHM'] <= 0.799) , 'DesarrolloNum'] = '4'
datosMun.loc[datosMun['IDHM'] >= 0.80 , 'DesarrolloNum'] = '5'

In [11]:
col_minimercados = np.argwhere(datosMun.columns.isin(['MiniMercDemais','MiniMercEPP', 'MiniMercME', 'MiniMercMEI'])).ravel()
col_supermercados = np.argwhere(datosMun.columns.isin(['SupermercadoDemais','SupermercadoEPP', 'SupermercadoME', 'SupermercadoMEI'])).ravel()
col_conveniencias = np.argwhere(datosMun.columns.isin(['ConvenienciaDemais', 'ConvenienciaEPP', 'ConvenienciaME','ConvenienciaMEI'])).ravel()
col_hipermercados =  np.argwhere(datosMun.columns.isin(['HiperDemais', 'HiperEPP'])).ravel()
col_atacados = np.argwhere(datosMun.columns.isin(['AtacadoDemais', 'AtacadoEPP'])).ravel()

In [12]:
datosMun['Minimercados'] = datosMun.iloc[:,col_minimercados].sum(axis=1)
datosMun['Supermercados'] = datosMun.iloc[:,col_supermercados].sum(axis=1)
datosMun['Conveniencias'] = datosMun.iloc[:,col_conveniencias].sum(axis=1)
datosMun['Hipermercados'] = datosMun.iloc[:,col_hipermercados].sum(axis=1)
datosMun['Atacados'] = datosMun.iloc[:,col_atacados].sum(axis=1)

In [13]:
datosMun.loc[(datosMun.Hipermercados + datosMun.Atacados) == 0, 'TipoComercios'] = 'Sin Hipermercados ni Atacados'
datosMun.loc[((datosMun.Hipermercados + datosMun.Atacados) >0) & ((datosMun.Hipermercados + datosMun.Atacados) <= 5), 'TipoComercios' ] = '< 5 Atacados e Hipermercados'
datosMun.loc[((datosMun.Hipermercados + datosMun.Atacados) >5) & ((datosMun.Hipermercados + datosMun.Atacados) <= 10), 'TipoComercios' ] = '< 10 Atacados e Hipermercados'
datosMun.loc[(datosMun.Hipermercados + datosMun.Atacados) >10 , 'TipoComercios' ] = '> 10 Atacados e Hipermercados'
datosMun.loc[(datosMun.Hipermercados + datosMun.Atacados) == 0, 'TipoComerciosNum'] = '1'
datosMun.loc[((datosMun.Hipermercados + datosMun.Atacados) >0) & ((datosMun.Hipermercados + datosMun.Atacados) <= 5), 'TipoComerciosNum' ] = '2'
datosMun.loc[((datosMun.Hipermercados + datosMun.Atacados) >5) & ((datosMun.Hipermercados + datosMun.Atacados) <= 10), 'TipoComerciosNum' ] = '3'
datosMun.loc[(datosMun.Hipermercados + datosMun.Atacados) >10 , 'TipoComerciosNum' ] = '4'

In [14]:
datosMun = datosMun.drop(['VAAgro', 'VAIndustria','VAServicos', 'VAAdmPublica','VATotal', 'Impostos', 'PBI','Atividade secundaria', 'Atividade terciaria', 'Area_km2',
                          'Habitantes_km2', 'PorcEscolarizacion6a14', 'Mortalidad_Infantil', 'ReceitaPrefeitura', 'DespesasPrefeitura','Mortalidad_Infantil', 'ReceitaPrefeitura', 'DespesasPrefeitura','HiperDemais', 'HiperEPP', 'HiperME', 'HiperMEI', 'MiniMercDemais', 'MiniMercEPP', 'MiniMercME', 'MiniMercMEI', 'SupermercadoDemais', 'SupermercadoEPP', 'SupermercadoME', 'SupermercadoMEI', 'ConvenienciaDemais', 'ConvenienciaEPP', 'ConvenienciaME', 'ConvenienciaMEI', 'AtacadoDemais', 'AtacadoEPP', 'AtacadoME', 'AtacadoMEI'
                          ], axis=1)

Union con la base de datos de ventas y creacion del archivo CSV

In [15]:
ventas_tot_mun = pd.merge(ventasLM, datosMun)

In [16]:
ventas_tot_mun.to_csv("D:/Zaeli/Analise Cliente Produto Comercial/Analisis Datos/VentasporLineaMnicipio.csv")